In [1]:
import pandas as pd
import numpy as np
from openai import OpenAI

data = pd.read_csv('../Datasets/all-claims.csv')

In [2]:
OPENAI_API_KEY = ##
client = OpenAI(api_key=OPENAI_API_KEY)

def chat(system_msg, user_msg, model="gpt-4o"):
  system_msg = [{"role": "system", "content": system_msg}]
  user_assistant_msgs = [{"role": "assistant", "content": user_msg} ]
  msgs = system_msg + user_assistant_msgs
  response = client.chat.completions.create(model=model,messages=msgs)
  return response.choices[0].message.content

In [19]:
def create_augmented_headlines(claim):
    res = chat("Create exactly 10 realistic news headlines that contain the following claim. Return just the headlines, separated by semicolons; do not number them.", claim).split(";")
    if len(res) < 10:
        print("Error: ", res)
        return None
    return res

In [ ]:
augmented_data = pd.DataFrame(columns=["text", "headline", "rating"])
for i, row in data.iterrows():
    text = row["text"]
    headlines = create_augmented_headlines(text)
    if headlines is None:
        print("Error: ", text)
        continue
    new_df = pd.DataFrame({"text": [text]*len(headlines), "headline": headlines, "rating": [row["rating"]]*len(headlines)})
    augmented_data = pd.concat([augmented_data, new_df], ignore_index=True)
augmented_data

In [25]:
augmented_data.to_csv("augmented_headliens.csv", index=False)